In [1]:
import numpy as np
def matmul(a, b):
    n = a.shape[0]
    k = a.shape[1]
    m = b.shape[1]  
    c = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            for s in range(k):
                c[i, j] += a[i, s] * b[s, j]
                
    return c

def matmul_isj(a, b):
    n = a.shape[0]
    k = a.shape[1]
    m = b.shape[1]  
    c = np.zeros((n, m))
    for i in range(n):
        for s in range(k):
            for j in range(m):
                c[i, j] += a[i, s] * b[s, j]
                
    return c

In [2]:
import numpy as np
from numba import jit # Just-in-time compiler for Python, see http://numba.pydata.org 

@jit(nopython=True)
def numba_matmul(a, b):
    n = a.shape[0]
    k = a.shape[1]
    m = b.shape[1]
    c = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            for s in range(k):
                c[i, j] += a[i, s] * b[s, j]
    return c

@jit(nopython=True)
def numba_matmul_isj(a, b):
    n = a.shape[0]
    k = a.shape[1]
    m = b.shape[1]
    c = np.zeros((n, m))
    for i in range(n):
        for s in range(k):
            mult = a[i, s]
            for j in range(m):
                c[i, j] += mult * b[s, j]
    return c

In [3]:
n = 500
a = np.random.randn(n, n)
b = np.random.randn(n, n)

# %timeit matmul(a, b)
# %timeit matmul_isj(a, b)
%timeit numba_matmul(a, b)
%timeit numba_matmul_isj(a, b)
%timeit np.dot(a, b)

1 loop, best of 5: 168 ms per loop
The slowest run took 7.12 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 38.2 ms per loop
100 loops, best of 5: 7.87 ms per loop


In [4]:
A = np.array([[1, 2, 3], [5, 2, 7]], order="C")

In [5]:
A.dtype

dtype('int64')

In [6]:
import torch

In [7]:
A = torch.from_numpy(a)
B = torch.from_numpy(b)

In [8]:
# How to call BLAS/LAPACK functions from torch 
torch.__config__.show()

'PyTorch built with:\n  - GCC 7.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.1\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86\n  - CuDNN 8.0.5\n  - Magma 2.5.2\n  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.1, CUDNN_VERSION=8.0.5, CXX_COMPILER=/opt/rh/devtoolset-7/root/usr/bin/c++, CXX_FLAGS= -Wno-deprecated -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -fopenmp -DNDEBUG -DUSE_KINETO -DUSE_FBGEMM -DUSE_QNNPACK -DUSE_PYTORCH_QNNPACK -DUSE_XNNP

Search section BLAS and LAPACK Operations in https://pytorch.org/docs/1.10.0/torch.html 

You will have some strange acronyms like
- ```geqrf```
- ```ger```
- ```ormqr```
- etc...

These names come from native LAPACK package and are standard for implementation of more complicated algorityhmes from the simple primitives. 

In [9]:
%timeit A @ B

The slowest run took 10.87 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 5: 6.76 ms per loop


In [10]:
A_gpu = A.to("cuda")
B_gpu = B.to("cuda")
%timeit A_gpu @ B_gpu

The slowest run took 177.63 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 366 µs per loop


In [11]:
print(A)

tensor([[ 1.2076,  0.2286, -0.1727,  ..., -1.3513,  0.6674,  0.7101],
        [-0.5689,  0.7661,  0.6888,  ..., -1.0185,  0.2321,  1.0647],
        [-0.5084,  1.0221,  0.5718,  ..., -1.0592, -2.0717,  0.5223],
        ...,
        [-0.1754, -0.1383,  1.0748,  ..., -0.0469, -0.1058, -0.9498],
        [-0.6619,  0.5752, -0.1461,  ...,  1.6014, -1.0753,  1.8088],
        [ 0.6649,  0.0372, -1.6637,  ...,  0.2114,  0.2700, -0.6503]],
       dtype=torch.float64)


In [12]:
t = torch.randn((n, n))
print(t)

tensor([[ 0.8519, -0.8576,  1.4654,  ..., -0.3948, -1.4546, -1.8130],
        [ 0.5388,  0.3280, -1.3981,  ..., -2.4956,  1.3326,  0.1441],
        [-1.4787, -0.4034,  0.9097,  ..., -1.2501,  0.2794,  0.7110],
        ...,
        [ 0.8301, -2.0607, -0.6245,  ..., -0.5105, -1.2650, -1.3490],
        [-1.5284,  0.4654,  1.0024,  ..., -0.1602,  1.3207, -0.6443],
        [-1.3054,  0.6352, -0.6033,  ...,  0.4865, -1.4938, -1.9792]])


In [13]:
t.dtype

torch.float32

In [18]:
import jax
import jax.numpy as jnp

from jax.config import config
config.update("jax_enable_x64", True)

A_jax = jnp.array(a)
B_jax = jnp.array(b)
%timeit (A_jax @ B_jax).block_until_ready()

1000 loops, best of 5: 457 µs per loop


In [19]:
A_jax.device_buffer.device()

GpuDevice(id=0, process_index=0)

In [20]:
A_jax.dtype

dtype('float64')